In [ ]:
import numpy as np
import nibabel as nb
import matplotlib.pyplot as plt
from matplotlib_surface_plotting import plot_surf
import trimesh
import io
from PIL import Image
from sklearn import neighbors

In [ ]:
!which python

In [ ]:
def get_layer(V, F, surface_index, n_surfs):
    """Retrieve the vertices and faces corresponding to a specific surface layer.

    Args:
        V (numpy.ndarray): Array of vertices representing the entire mesh.
        F (numpy.ndarray): Array of faces representing the entire mesh.
        surface_index (int): Index of the surface layer to retrieve.
        n_surfs (int): Total number of surface layers in the mesh.

    Returns:
        tuple: A tuple containing the surface vertices and faces.
        
    """
    num_vertices_per_surface = len(V) // n_surfs
    start_vertex_index = surface_index * num_vertices_per_surface
    end_vertex_index = start_vertex_index + num_vertices_per_surface

    mesh = trimesh.Trimesh(V, F, process=False, validate=False)
    mask=np.zeros(V.shape[0])
    mask[start_vertex_index:end_vertex_index]=1
    mesh.update_vertices(mask.astype(bool))
    surface_vertices=mesh.vertices
    surface_faces=mesh.faces
    return surface_vertices, surface_faces

In [ ]:
peak_idx=5618

In [ ]:
multilayer_fname='/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.link_vector.nodeep.gii'
# Load multilayer surface
multilayer_surf=nb.load(multilayer_fname);
multilayer_vertices, multilayer_faces = multilayer_surf.darrays[0].data, multilayer_surf.darrays[1].data

n_surfs=11
verts_per_surf=int(multilayer_vertices.shape[0]/n_surfs)
print('{} surfaces, {} vertices per surface'.format(n_surfs,verts_per_surf))

for i in range(n_surfs):
    pial_vertices, pial_faces = get_layer(multilayer_vertices, multilayer_faces, i, n_surfs)
    rotate = [0]
    x_rotate = 0
    fig = plt.figure(facecolor='white')
    # Plot inversion result
    ax = plt.subplot(1, 1, 1, xlim=[-.98, +.98], ylim=[-.98, +.98],
                     aspect=1, frameon=False,
                     xticks=[], yticks=[])
    overlay = np.zeros(pial_vertices.shape[0])
    overlay[0] = 10
    plot_surf(vertices=pial_vertices, faces=pial_faces, overlay=overlay, rotate=rotate, x_rotate=x_rotate,
              ax=ax, colorbar=False, cmap='jet', alpha_colour=(overlay > 0).astype(int), vmin=0, vmax=4,
              transparency=.5, coords=[pial_vertices[peak_idx,:]], coord_size=0.01)    
    


In [ ]:
multilayer_fname='/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.link_vector.gii'
# Load multilayer surface
multilayer_surf=nb.load(multilayer_fname);
multilayer_vertices, multilayer_faces = multilayer_surf.darrays[0].data, multilayer_surf.darrays[1].data

n_surfs=11
verts_per_surf=int(multilayer_vertices.shape[0]/n_surfs)
print('{} surfaces, {} vertices per surface'.format(n_surfs,verts_per_surf))

for i in range(n_surfs):
    pial_vertices, pial_faces = get_layer(multilayer_vertices, multilayer_faces, i, n_surfs)
    rotate = [0]
    x_rotate = 0
    fig = plt.figure(facecolor='white')
    # Plot inversion result
    ax = plt.subplot(1, 1, 1, xlim=[-.98, +.98], ylim=[-.98, +.98],
                     aspect=1, frameon=False,
                     xticks=[], yticks=[])
    overlay = np.zeros(pial_vertices.shape[0])
    overlay[0] = 10
    plot_surf(vertices=pial_vertices, faces=pial_faces, overlay=overlay, rotate=rotate, x_rotate=x_rotate,
              ax=ax, colorbar=False, cmap='jet', alpha_colour=(overlay > 0).astype(int), vmin=0, vmax=4,
              transparency=.5, coords=[pial_vertices[peak_idx,:]], coord_size=0.01)    
    


In [ ]:
pial_p = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/pial.ds.gii"
white_p = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/white.ds.gii"
pial_ = "/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/white.ds.link_vector.nodeep.gii"


onelayer_surf = nb.load(pial_);
onelayer_surf_vertices, onelayer_surf_faces = onelayer_surf.darrays[0].data, onelayer_surf.darrays[1].data
vertices_n = onelayer_surf_vertices.shape[0]
rotate = [0]
x_rotate = 0
fig = plt.figure(facecolor='white')
ax = plt.subplot(
    1, 1, 1, xlim=[-.98, +.98], ylim=[-.98, +.98],
    aspect=1, frameon=False, xticks=[], yticks=[]
)
overlay = np.zeros(vertices_n)
overlay[0] = 10
plot_surf(
    vertices=onelayer_surf_vertices, faces=onelayer_surf_faces,
    overlay=overlay, rotate=rotate, x_rotate=x_rotate,
    ax=ax, colorbar=False, cmap='jet', alpha_colour=(overlay > 0).astype(int),
    vmin=0, vmax=4, transparency=.5, coords=[onelayer_surf_vertices[peak_idx,:]], coord_size=0.01
)

In [ ]:
deep_surf = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/pial.ds.link_vector.gii");
deep_surf_vertices, deep_surf_faces = deep_surf.darrays[0].data, deep_surf.darrays[1].data

mesh = trimesh.Trimesh(deep_surf_vertices, deep_surf_faces)
mesh.show(viewer="notebook")

In [ ]:
mesh

In [ ]:
nodeep_surf = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/pial.ds.link_vector.nodeep.gii");
nodeep_surf_vertices, nodeep_surf_faces = nodeep_surf.darrays[0].data, nodeep_surf.darrays[1].data

mesh = trimesh.Trimesh(nodeep_surf_vertices, nodeep_surf_faces)
mesh.show(viewer="notebook")

In [ ]:
tree = neighbors.KDTree(nodeep_surf_vertices, leaf_size=20)
ixes = []
coords = []
for d_vs in deep_surf_vertices:
    distance, index = tree.query(d_vs.reshape(1, -1), k=1)
    if distance[0] > 0:
        ixes.append(index[0])
        coords.append(d_vs.reshape(1, -1))

In [ ]:
coords

In [ ]:
n_surf=11
ml_deep = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.gii");
ml_deep_vertices, ml_deep_faces = ml_deep.darrays[0].data, ml_deep.darrays[1].data
ml_deep_vertices = np.split(ml_deep_vertices, n_surf, axis=0)
ml_deep_faces = np.split(ml_deep_faces, n_surf, axis=0)

ml_nodeep = nb.load("/home/common/bonaiuto/multiburst/derivatives/processed/sub-001/multilayer_11/multilayer_11.ds.link_vector.nodeep.gii");
ml_nodeep_vertices, ml_nodeep_faces = ml_nodeep.darrays[0].data, ml_nodeep.darrays[1].data
ml_nodeep_vertices = np.split(ml_nodeep_vertices, n_surf, axis=0)
#ml_nodeep_faces = np.split(ml_nodeep_faces, n_surf, axis=0)

In [ ]:
mesh = trimesh.Trimesh(ml_deep_vertices[0], ml_deep_faces[0])
mesh.show(viewer="notebook")

In [ ]:
mesh = trimesh.Trimesh(ml_deep_vertices[0], ml_deep_faces[0])
mesh.show(viewer="notebook")

In [ ]:
tree = neighbors.KDTree(ml_deep_vertices[0], leaf_size=20)
ixes = []
for i in coords:
    distance, index = tree.query(i, k=1)
    ixes.append(index[0][0])
#     print(distance[0][0], index[0][0])

In [ ]:
mesh = trimesh.Trimesh(ml_deep_vertices[1], ml_deep_faces[1], process=False, validate=False)
mesh.show(viewer="notebook")